In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ 
# and enter as the spark version
# For example:
# spark_version = 'spark-3.3.0'

spark_version = 'spark-3.3.0'
os.environ['SPARK_VERSION'] = spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Install Postgres Driver
!wget https://jdbc.postgresql.org/download/postgresql-42.5.0.jar

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [913 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:13 

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Vine_Review_Analysis_Instruments").config("spark.driver.extraClassPath","/content/postgresql-42.5.0.jar").getOrCreate()

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Musical_Instruments_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Musical_Instruments_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   45610553| RMDCHWD0Y5OZ9|B00HH62VB6|     618218723|AGPtek® 10 Isolat...|Musical Instruments|          3|            0|          1|   N|                N|         Three Stars|Works very good, ...|2015-08-31 00:00:00|
|         US|   14640079| RZSL0BALIYUNU|B003LRN53I|     986692292|Sennheiser

In [4]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RMDCHWD0Y5OZ9|          3|            0|          1|   N|                N|
| RZSL0BALIYUNU|          5|            0|          0|   N|                Y|
| RIZR67JKUDBI0|          3|            0|          1|   N|                Y|
|R27HL570VNL85F|          5|            0|          0|   N|                Y|
|R34EBU9QDWJ1GD|          5|            0|          0|   N|                Y|
|R1WCUI4Z1SIQEO|          5|            0|          0|   N|                N|
| RL5LNO26GAVJ1|          2|            3|          4|   N|                Y|
|R3GYQ5W8JHP8SB|          5|            0|          0|   N|                Y|
|R30SHYQXGG5EYC|          5|            0|          0|   N|                Y|
|R14YLXA56NP51I|          5|            1|          1|   N|     

In [5]:
# Create a new DataFrame to retrieve all rows where total_votes count is equal to or greater than 20
vine_vote20_df = vine_df.filter(vine_df.total_votes > 20)
vine_vote20_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2243Y3OD8U6KQ|          5|           47|         61|   N|                N|
|R2TGT0CDTCAAHW|          5|           21|         23|   N|                Y|
| RX4D22YSXEF4P|          1|           37|         38|   N|                Y|
|R3FL2NTLFUSPTQ|          5|           33|         37|   N|                N|
|R3QTP3YNZXAPPF|          3|           23|         24|   N|                Y|
|R36V6V42VN5AS5|          5|           34|         37|   N|                Y|
|R27LZWE27BJPOB|          5|           22|         23|   N|                N|
|  RMRD6SMF2AUQ|          3|            4|         21|   N|                N|
| RMPCXKWX3T57Y|          1|            1|         73|   N|                N|
|R1P7GJ0IN2BRNH|          5|           37|         37|   N|     

In [6]:
# Create a new DataFrame where helpful_votes / total_votes is greater than 50%
vine_helpfulTotal_df = vine_vote20_df.filter(vine_vote20_df.helpful_votes / vine_vote20_df.total_votes >= 0.5)
vine_helpfulTotal_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2243Y3OD8U6KQ|          5|           47|         61|   N|                N|
|R2TGT0CDTCAAHW|          5|           21|         23|   N|                Y|
| RX4D22YSXEF4P|          1|           37|         38|   N|                Y|
|R3FL2NTLFUSPTQ|          5|           33|         37|   N|                N|
|R3QTP3YNZXAPPF|          3|           23|         24|   N|                Y|
|R36V6V42VN5AS5|          5|           34|         37|   N|                Y|
|R27LZWE27BJPOB|          5|           22|         23|   N|                N|
|R1P7GJ0IN2BRNH|          5|           37|         37|   N|                Y|
|R2J0ZZGFXKM8KR|          2|           21|         22|   N|                Y|
|R238ZSG6TSUBNX|          5|           27|         27|   N|     

# Analysis of Reviews Paid for by Vine

In [7]:
# Create a new DataFrame where the review was written as part of the paid Vine program
vine_paid_df = vine_helpfulTotal_df.filter(vine_helpfulTotal_df.vine == 'Y')
vine_paid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1R9RU7JW0MFR2|          4|           20|         23|   Y|                N|
|R19EFYNN3W8Q07|          5|           26|         32|   Y|                N|
|R34DJ1R8AEU0SG|          5|           29|         35|   Y|                N|
|R25P5CXK5L9RHF|          5|          146|        161|   Y|                N|
|R2E9VZB3I4LSN5|          5|           55|         59|   Y|                N|
| RKYLHZL7EPELX|          4|           19|         25|   Y|                N|
|R1U13EKGQD3ZE6|          5|           22|         25|   Y|                N|
| RYW05F1MUEF01|          5|           87|        102|   Y|                N|
|R2SW4NXNO7HZJ5|          4|           28|         33|   Y|                N|
|R2016NFLSUR97Y|          2|           26|         37|   Y|     

In [18]:
# Total number of reviews paid for by Vine
vine_paid_df.count()

59

In [21]:
# DataFrame of reviews paid for by Vine that had a 5star rating
vine_paid_star_df = vine_paid_df.filter(vine_paid_df.star_rating == 5)
vine_paid_star_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R19EFYNN3W8Q07|          5|           26|         32|   Y|                N|
|R34DJ1R8AEU0SG|          5|           29|         35|   Y|                N|
|R25P5CXK5L9RHF|          5|          146|        161|   Y|                N|
|R2E9VZB3I4LSN5|          5|           55|         59|   Y|                N|
|R1U13EKGQD3ZE6|          5|           22|         25|   Y|                N|
| RYW05F1MUEF01|          5|           87|        102|   Y|                N|
|R13PPTBWWV10Q6|          5|           19|         25|   Y|                N|
|R2A3FFNJTDUBMV|          5|           20|         27|   Y|                N|
|R3QQG4HUB214TM|          5|          111|        128|   Y|                N|
|R3J9OSJNJB7KGM|          5|           70|         80|   Y|     

In [22]:
# Total number of reviews paid for by Vine with a 5star rating
vine_paid_star_df.count()

34

In [25]:
# Percentage of reviews not paid for by Vine with 5star ratings
vine_paid_star_df.count() / vine_paid_df.count() * 100

57.6271186440678

# Analysis of Reviews **Not** Paid for By Vine

In [10]:
# Create a new DataFrame where the review was written as part of the paid Vine program
vine_notPaid_df = vine_helpfulTotal_df.filter(vine_helpfulTotal_df.vine == 'N')
vine_notPaid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2243Y3OD8U6KQ|          5|           47|         61|   N|                N|
|R2TGT0CDTCAAHW|          5|           21|         23|   N|                Y|
| RX4D22YSXEF4P|          1|           37|         38|   N|                Y|
|R3FL2NTLFUSPTQ|          5|           33|         37|   N|                N|
|R3QTP3YNZXAPPF|          3|           23|         24|   N|                Y|
|R36V6V42VN5AS5|          5|           34|         37|   N|                Y|
|R27LZWE27BJPOB|          5|           22|         23|   N|                N|
|R1P7GJ0IN2BRNH|          5|           37|         37|   N|                Y|
|R2J0ZZGFXKM8KR|          2|           21|         22|   N|                Y|
|R238ZSG6TSUBNX|          5|           27|         27|   N|     

In [11]:
# Total number of reviews not paid for by Vine
vine_notPaid_df.count()

13480

In [12]:
# DataFrame of reviews not paid for by Vine that had a 5star rating
vine_notPaid_star_df = vine_notPaid_df.filter(vine_notPaid_df.star_rating == 5)
vine_notPaid_star_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2243Y3OD8U6KQ|          5|           47|         61|   N|                N|
|R2TGT0CDTCAAHW|          5|           21|         23|   N|                Y|
|R3FL2NTLFUSPTQ|          5|           33|         37|   N|                N|
|R36V6V42VN5AS5|          5|           34|         37|   N|                Y|
|R27LZWE27BJPOB|          5|           22|         23|   N|                N|
|R1P7GJ0IN2BRNH|          5|           37|         37|   N|                Y|
|R238ZSG6TSUBNX|          5|           27|         27|   N|                Y|
| RFL3TJED8C240|          5|           13|         24|   N|                N|
|R3D4P7A8Q1C70F|          5|           22|         27|   N|                Y|
|R3JNDLZ3KACRLG|          5|           29|         33|   N|     

In [13]:
# Total number of reviews not paid for by Vine with a 5star rating
vine_notPaid_star_df.count()

7678

In [14]:
# Percentage of reviews not paid for by Vine with 5star ratings
vine_notPaid_star_df.count() / vine_notPaid_df.count() * 100

56.95845697329377